# [Advent of Code 2015](https://adventofcode.com/2015)
Python solutions to the 2015 Christmas puzzles.

First, import standard libraries we'll use throughout.

In [1]:
import collections
import dataclasses
import functools
import itertools as its
import json
import math
import operator
import random
import re
import string
import typing
from pprint import pprint

Next, some general purpose helper functions for reading and parsing inputs etc.

In [2]:
def day(day_n):
    'Load the input for `day_n`'
    return open(f'input/{day_n}').read()

def ilen(xs):
    'Return the length of an iterable'
    return sum(1 for _ in iter(xs))

def last(xs):
    'Return the last element of an iterable sequence.'
    xs = iter(xs)
    return collections.deque(xs, maxlen=1)[0]

def ints(s):
    'Return a tuple of integer tokens in `s`.'
    return tuple(int(x) for x in re.findall(r'[-+]?\d+', s))

def array(data):
    'Convert the data into a tuple of int-tuples.'
    return tuple(ints(line) for line in data.splitlines())

def swap(xs, i, j):
    'Swap items in a sequence.'
    xs[i], xs[j] = xs[j], xs[i]

def nth(xs, n):
    'Return the nth element of a sequence.'
    return next(its.islice(xs, n, None))

def take(xs, n=None):
    '''Return (up to) first `n` items of a sequence
    or the whole sequence if `n` is `None`
    '''
    return list(xs) if n is None else [x for x, _ in zip(xs, range(n))]

def overlapping(xs, n):
    '''Generate overlapping n-tuples from `xs`

    overlapping('ABCDEFG', 3) --> ABC BCD CDE DEF EFG
    '''
    if isinstance(xs, collections.abc.Sequence):
        yield from (xs[i:i+n] for i in range(len(xs) + 1 - n))
    else:
        result = collections.deque(maxlen=n)
        for x in xs:
            result.append(x)
            if len(result) == n:
                yield tuple(result)
                
def pairwise(iterable):
    's -> (s0,s1), (s1,s2), (s2, s3), ...'
    return overlapping(iterable, 2)

def unfold(fn, x):
    while True:
        yield x
        x = fn(x)

def chunked(xs, n):
    'Collect `xs` into chunks of length `n`'
    return zip(*[iter(xs)] * n)

def manhattan_distance(P, Q):
    'Return the city block distance from `P` to `Q`'
    return sum(math.fabs(p-q) for p,q in zip(P, Q))

def neighbours4(x, y):
    'Return the horizontal and vertical neighbours of `(x, y)`'
    return (x,y-1),(x-1,y),(x+1,y),(x,y+1)

def neighbours8(x, y):
    'Return the horizontal, vertical and diagonal neighbours of `(x, y)`'
    return ((x-1,y-1),(x-1,y),(x-1,y+1),
            (x,y-1),(x,y+1),
            (x+1,y-1),(x+1,y),(x+1,y+1))

def unique(xs):
    'Return `True` iff no elements of `xs` are repeated'
    return len(xs) == len(set(xs))

def mapt(f, xs):
    return tuple(map(f, xs))

def atom(s):
    try:
        return int(s)
    except ValueError:
        return s

def id_(x):
    return x
    
def render_grid(grid, fill_ch=' '):
    '''Convert `grid` into a printable form
    
    grid - maps 2D points to single chars
    fill_ch - used for chars not in the grid
    '''
    xlo, xhi = min(map(first, grid)), max(map(first, grid))
    ylo, yhi = min(map(second, grid)), max(map(second, grid))
    return ncat(cat(grid.get((x,y), fill_ch)
                    for x in range(xlo, xhi+1))
                for y in range(ylo, yhi+1))

def draw_grid(grid, fill_ch=' '):
    print(render_grid(grid, fill_ch))       

flatten = its.chain.from_iterable
cat = ''.join
ncat = '\n'.join
first = operator.itemgetter(0)
second = operator.itemgetter(1)

## [Day 1](https://adventofcode.com/2015/day/1) Not Quite Lisp

The input is a string of parentheses which correspond to Santa going up and down in a building:
> An opening parenthesis, (, means he should go up one floor, and a closing parenthesis, ), means he should go down one floor.

Part one asks what floor Santa ends up on.

In [3]:
day1 = day(1)
day1.count('(') - day1.count(')')

138

For part two we're asked when Santa first visits the basement, i.e. floor `-1`.

In [4]:
floor = 0
steps = enumerate(day1, 1)
while floor != -1:
    t, c = next(steps)
    floor += {'(': 1, ')':-1}[c]
t    

1771

## [Day 2](https://adventofcode.com/2015/day/2) I Was Told There Would Be No Math

We have to calculate how much wrapping paper is required for a set of parcels. Each line of the input gives a parcel's dimensions, e.g. `3x11x24`.

In [5]:
def wrapping_paper(*sides):
    # Enough to cover the parcel + smallest side excess
    a, b, c = sorted(sides)
    return 3*a*b + 2*(b*c + c*a)

sum(wrapping_paper(*ints(s)) for s in day(2).splitlines())

1588178

Part two asked how much ribbon is needed for the parcels.

In [6]:
def ribbon(*sides):
    # Perimeter of smallest side + parcel volume for bow
    a, b, c = sorted(sides)
    return 2*(a + b) + a*b*c

sum(ribbon(*ints(s)) for s in day(2).splitlines())

3783758

## [Day 3](https://adventofcode.com/2015/day/3) Perfectly Spherical Houses in a Vacuum

A bit like Day 1, a string of characters corresponds to steps, this time in 2D, using the ascii arrows `<>^v`. How many positions in the grid do we visit? Let's use the complex plane as our map, so taking a step is done by adding the step direction.

In [7]:
def traverse_grid(steps, pos=0):
    'Yield positions visited by Santa'
    yield pos
    for s in steps:
        pos += {'^':-1j, '>':1, 'v':1j, '<':-1}[s]
        yield pos

len(set(traverse_grid(day(3))))

2572

For part two, Santa and Robo-Santa take steps alternately.

In [8]:
def traverse_grid(steps, n_santas=2):
    'Yield positions visited by the Santas'
    santas = [0] * n_santas
    yield from santas
    for s, i in zip(steps, its.cycle(list(range(n_santas)))):
        santas[i] += {'^':-1j, '>':1, 'v':1j, '<':-1}[s]
        yield santas[i]

assert len(set(traverse_grid('^v^v^v^v^v'))) == 11
len(set(traverse_grid(day(3))))

2631

In [9]:
# Note: the part two solution works for part one
assert len(set(traverse_grid(day(3), 1))) == 2572

## [Day 4](https://adventofcode.com/2015/day/4) The Ideal Stocking Stuffer

We're looking for a MD5 hash whose hexadecimal representation starts with a fixed pattern.

In [10]:
import hashlib

day5 = 'iwrupvqb'

assert next(n for n in its.count(1)
            if hashlib.md5(f'abcdef{n}'.encode())
            .hexdigest().startswith('00000')) == 609043

next(n for n in its.count(1)
     if hashlib.md5(f'{day5}{n}'.encode())
     .hexdigest().startswith('00000'))

346386

In [11]:
# Part 2 asks for a hexdigest starting with 6 zeros
next(n for n in its.count(1)
     if hashlib.md5(f'{day5}{n}'.encode())
     .hexdigest().startswith('000000'))

9958218

## [Day 5](https://adventofcode.com/2015/day/5) Doesn't He Have Intern-Elves For This?

We have to categorise strings as "naughty" or "nice", according to some rules.

In [12]:
import string
vowels = set('aeiou')
XX = {x+x for x in string.ascii_lowercase}


def nice(s, must_have=XX, must_not_have={'ab', 'cd', 'pq', 'xy'}):
    'Return `True` iff x is "nice"'
    return (sum(1 for x in s if x in vowels) >= 3 and
            any(x in s for x in must_have) and
            not any(x in s for x in must_not_have))

assert nice('ugknbfddgicrmopn')
assert nice('aaa')
assert not nice('jchzalrnumimnmhp') or nice('haegwjzuvuyypxyu') or nice('dvszwmarrgswjxmb')
sum(map(nice, day(5).splitlines()))

238

Part two changes the rules.

In [13]:
def repeated_pair(s):
    'Return `True` iff s contains the same adjacent pair in 2 non-overlapping locations'
    N = len(s)
    return any(s[i:i+2] == s[j:j+2]
               for i,j in its.combinations(range(N-1), 2) if j - i >= 2)

def separated_repeat(s):
    'Return `True` iff s contains a letter which repeats separated by exactly one letter'
    return any(s[i] == s[i+2] for i in range(len(s)-2))

def nice2(s):
    return repeated_pair(s) and separated_repeat(s)

assert nice2('qjhvhtzxzqqjkmpb')
assert nice2('xxyxx')
assert not nice2('uurcxstgmygtbstg')
assert not nice2('ieodomkazucvgmuy')
sum(map(nice2, day(5).splitlines()))

69

## [Day 6](https://adventofcode.com/2015/day/6) Probably a Fire Hazard

We have been given instructions to toggle lights arranged in a 1000 x 1000 grid. The instructions are as shown: 

In [14]:
!head -n 5 input/6

toggle 461,550 through 564,900
turn off 370,39 through 425,839
turn off 464,858 through 833,915
turn off 812,389 through 865,874
turn on 599,989 through 806,993


In [15]:
def toggle_lights(instructions, W=1000, H=1000):
    grid = {p: 0 for p in its.product(range(W), range(H))}
    def     on(p): grid[p] = 1
    def    off(p): grid[p] = 0
    def toggle(p): grid[p] = not grid[p]

    for op in instructions:
        t,l,b,r = ints(op)
        if   'toggle' in op: fn = toggle
        elif 'on'     in op: fn = on
        else               : fn = off
        for p in its.product(range(t, b+1), range(l, r+1)):
            fn(p)
    return grid

sum(toggle_lights(day(6).splitlines()).values())

543903

Part two provides a second interpretation for the toggle, on, off operations. I rewrote toggle_lights to take these functions as a parameter.

In [16]:
def toggle_lights(instructions, fns, W=1000, H=1000):
    grid = {p: 0 for p in its.product(range(W), range(H))}
    for op in instructions:
        t,l,b,r = ints(op)
        fn = fns[next(s for s in ('on', 'off', 'toggle') 
                      if s in op)]
        for p in its.product(range(t, b+1), range(l, r+1)):
            fn(grid, p)
    return grid

def on(grid, p): grid[p] += 1
def off(grid, p): grid[p] = max(0, grid[p]-1)
def toggle(grid, p): grid[p] += 2

sum(toggle_lights(day(6).splitlines(), {'on':on, 'off':off, 'toggle':toggle}).values())

14687245

The solution for part two can also be used for part one.

In [17]:
def on(grid, p): grid[p] = 1
def off(grid, p): grid[p] = 0
def toggle(grid, p): grid[p] = not grid[p]

sum(toggle_lights(day(6).splitlines(), {'on':on, 'off':off, 'toggle':toggle}).values())

543903

## [Day 7](https://adventofcode.com/2015/day/7) Some Assembly Required

We're given the connections to build an electical circuit:

> Each wire has an identifier (some lowercase letters) and can carry a 16-bit signal (a number from 0 to 65535). A signal is provided to each wire by a gate, another wire, or some specific value. Each wire can only get a signal from one source, but can provide its signal to multiple destinations. A gate provides no signal until all of its inputs have a signal.

We're asked to run the circuit and find the signal ultimately provided to `a`.

We're given a simple circuit as an example, and told the final signals on the wires.

In [18]:
example_circuit_data = '''\
123 -> x
456 -> y
x AND y -> d
x OR y -> e
x LSHIFT 2 -> f
y RSHIFT 2 -> g
NOT x -> h
NOT y -> i
'''

example_signals = {
    'd':72, 'e':507 ,'f':492, 'g':114, 'h':65412,
    'i':65079, 'x':123, 'y':456
}

First, let's read in the input. 

In [19]:
# Create a dict mapping from operator name to operator function
import operator as ops
circuit_fns = {'AND':ops.and_, 'OR':ops.or_, 'LSHIFT':ops.lshift, 'RSHIFT':ops.rshift }

def load_circuit(circuit_data):
    ''' Return dict mapping wire to the value of that wire
    
    Note that the "value" is a function and the parameters to that function
    '''
    def load_wire(w):
        expr, _, wire = w.partition(' -> ')
        terms = mapt(atom, expr.split())
        if   len(terms) == 1: value = id_, terms[0]                # assign
        elif len(terms) == 2: value = lambda v: 0xffff-v, terms[1] # NOT
        else: value = circuit_fns[terms[1]], terms[0], terms[2]    # BINOP
        return wire, value

    return dict(load_wire(w) for w in circuit_data.splitlines())

load_circuit(example_circuit_data)

{'x': (<function __main__.id_(x)>, 123),
 'y': (<function __main__.id_(x)>, 456),
 'd': (<function _operator.and_(a, b, /)>, 'x', 'y'),
 'e': (<function _operator.or_(a, b, /)>, 'x', 'y'),
 'f': (<function _operator.lshift(a, b, /)>, 'x', 2),
 'g': (<function _operator.rshift(a, b, /)>, 'y', 2),
 'h': (<function __main__.load_circuit.<locals>.load_wire.<locals>.<lambda>(v)>,
  'x'),
 'i': (<function __main__.load_circuit.<locals>.load_wire.<locals>.<lambda>(v)>,
  'y')}

Now we can evaluate the circuit.

In [20]:
example_circuit = load_circuit(example_circuit_data)

def evaluate_wire(circuit, wire):
    'Return the signal emitted by `wire` for the given `circuit`'
    @functools.lru_cache(maxsize=None)
    def evaluate(wire):
        fn, *vals = circuit[wire]
        def evaluate_(v):
            return v if isinstance(v, int) else evaluate(v)
        return fn(*[evaluate_(v) for v in vals])
    return evaluate(wire)

# Check the worked example
for w, v in example_signals.items():
    assert evaluate_wire(example_circuit, w) == v

# We're asked for the final value of Wire a
circuit = load_circuit(day(7))
evaluate_wire(circuit, 'a')

956

Part two asks us to set wire `b` to this value of wire `a`, then recalculate the signal on wire `a`.

In [21]:
circuit['b'] = id_, 956
evaluate_wire(circuit, 'a')

40149

In [22]:
# For fun, render the circuit dependencies as a graph.
# This takes a while and the graph is huge!
# import graphviz

In [23]:
def circuit_graph(circuit, **k):
    g = graphviz.Digraph(**k)
    for v, args in circuit.items():
        g.node(v)
        g.edges([(v, str(a)) for a in args[1:]])
    return g

# g = circuit_graph(load_circuit(day(7)), format="png")

In [24]:
# g.render('circuit', view=True)  

## [Day 8](https://adventofcode.com/2015/day/8) Matchsticks

We have a list of strings. The strings may contains escape sequences `\\`, `\"` and `\xHh` (where H, h are hexadecimal characters). We are asked for the actual length of the strings minus the total length of their contents in memory.

Since these are valid Python string escape sequences we can get the answer directly.

In [25]:
def surplus_length(strings):
    return sum(len(s) - eval(f'len({s})') for s in strings.splitlines())

example_8 = '''\
""
"abc"
"aaa\\"aaa"
"\\x27"
'''

assert surplus_length(example_8) == 12

In [26]:
surplus_length(day(8))

1333

Part two asks us to go the other way. For example, `""` encodes as `"\"\""`, an increase from `2` characters to `5`.

In [27]:
def excess_length(s):
    return s.count('\\') + s.count('"') + 2

assert sum(map(excess_length, example_8.splitlines())) == 19

sum(map(excess_length, day(8).splitlines()))

2046

## [Day 9](https://adventofcode.com/2015/day/9) All in a Single Night

We're asked for the shortest route Santa can take to visit each location exactly once -- the shortest Hamiltonian path.

In [28]:
!cat input/9

AlphaCentauri to Snowdin = 66
AlphaCentauri to Tambi = 28
AlphaCentauri to Faerun = 60
AlphaCentauri to Norrath = 34
AlphaCentauri to Straylight = 34
AlphaCentauri to Tristram = 3
AlphaCentauri to Arbre = 108
Snowdin to Tambi = 22
Snowdin to Faerun = 12
Snowdin to Norrath = 91
Snowdin to Straylight = 121
Snowdin to Tristram = 111
Snowdin to Arbre = 71
Tambi to Faerun = 39
Tambi to Norrath = 113
Tambi to Straylight = 130
Tambi to Tristram = 35
Tambi to Arbre = 40
Faerun to Norrath = 63
Faerun to Straylight = 21
Faerun to Tristram = 57
Faerun to Arbre = 83
Norrath to Straylight = 9
Norrath to Tristram = 50
Norrath to Arbre = 60
Straylight to Tristram = 27
Straylight to Arbre = 81
Tristram to Arbre = 90


Inspecting the data, we can see there are just 7 destinations. A brute force solution will be fine.

In [29]:
def route_map(route_data):
    'Load the route data into a dict mapping route => distance'
    routes ={(w[0], w[2]): int(w[-1]) 
            for w in map(str.split, route_data.splitlines())}
    # Add the reverse routes
    return {**routes, **{(dst, src):d for (src, dst), d in routes.items()}}

def route_len(all_routes, route):
    return sum(all_routes[p] for p in pairwise(route))

def shortest_path(route_data, min_fn=min):
    routes = route_map(route_data)
    cities = {city for city, _ in routes}
    return min_fn(route_len(routes, route) 
                  for route in its.permutations(cities, len(cities)))

assert shortest_path('''\
London to Dublin = 464
London to Belfast = 518
Dublin to Belfast = 141
''') == 605

shortest_path(day(9))

141

Part two asks for the longest path. I edited the solution to part one to parameterise `min_fn`.

In [30]:
shortest_path(day(9), max)

736

## [Day 10](https://adventofcode.com/2015/day/10) Elves Look, Elves Say

Part one asks for the 40th look-say number after a supplied value.

In [31]:
input10 = '3113322113'

def look_say(seed):
    while True:
        yield seed
        seed = list(flatten([str(len(list(g))), v] for v, g in its.groupby(seed)))

xs = look_say('1')
take(xs, 4)

['1', ['1', '1'], ['2', '1'], ['1', '2', '1', '1']]

In [32]:
len(nth(look_say(input10), 40))

329356

In [33]:
len(nth(look_say(input10), 50))

4666278

## [Day 11](https://adventofcode.com/2015/day/11) Corporate Policy

Generate passwords sequentially, matching the security elf's policies.

In [34]:
alphabet = string.ascii_lowercase
input_11 = 'hxbxwxba'

def straight(xs, n=3):
    'Return `True` if xs has an increasing straight of length n'
    return any(xs[i:i+3] in alphabet for i in range(len(xs)-n))

def allowed(xs, prohibited="iol"):
    return not any(x in prohibited for x in xs)

def diffpairs(xs):
    return len({p for p in pairwise(xs) if p[0]==p[1]}) > 1

def good_password(pword, *rules):
    return all(rule(pword) for rule in rules)

rules = [straight, allowed, diffpairs]

assert not good_password('hijklmmn', *rules)
assert not good_password('abbceffg', *rules)
assert not good_password('abbcegjk', *rules)
assert good_password('abcdffaa', *rules)

In [35]:
def next_permutation(xs, next_elem, first_elem):
    '''Return the next permutation of the string `xs`
    
    - next_elem - a dict mapping chars to the next char
    - first_elem - the first char
    '''
    xs = list(xs)
    for i in range(len(xs)-1, -1, -1):
        xs[i] = next_elem.get(xs[i], first_elem)
        if xs[i] != first_elem:
            return cat(xs)

def good_passwords(pword, *rules):
    'Generate a sequence of good passwords, starting from the supplied password'
    next_elem = dict(zip(alphabet, alphabet[1:]))
    first_elem = alphabet[0]
    while True:
        pword = next_permutation(pword, next_elem, first_elem)
        if good_password(pword, *rules):
            yield pword

# Check the examples supplied in the question
assert next(good_passwords('abcdefgh', *rules)) == 'abcdffaa'
assert next(good_passwords('ghijklmn', *rules)) == 'ghjaabcc'

next(good_passwords(input_11, *rules))

'hxbxxyzz'

Part two asks for the permitted password.

In [36]:
nth(good_passwords(input_11, *rules), 1)

'hxcaabcc'

## [Day 12](https://adventofcode.com/2015/day/12) JSAbacusFramework.io

The first part of the puzzle asks us to sum up all numbers in a JSON document. Conveniently, there are no numbers embedded in strings, and we can simply match all numeric sequences in the document.

In [37]:
sum(ints(day(12)))

111754

For Part Two, we're asked:

> Ignore any object (and all of its children) which has any property with the value "red". Do this only for objects ({...}), not arrays ([...]).

In [38]:
def sum_non_red_numbers(o):
    if isinstance(o, int): 
        return o
    elif isinstance(o, str):
        return 0
    elif isinstance(o, list):
        return sum(sum_non_red_numbers(c) for c in o)
    elif 'red' in o.values():
        return 0
    else:
        return sum(sum_non_red_numbers(v) for v in o.values())

# Check worked examples
assert sum_non_red_numbers(json.loads('[1,2,3]')) == 6
assert sum_non_red_numbers(json.loads('[1,{"c":"red","b":2},3]')) == 4
assert sum_non_red_numbers(json.loads('{"d":"red","e":[1,2,3,4],"f":5}')) == 0
assert sum_non_red_numbers(json.loads('[1,"red",5]')) == 6

In [39]:
sum_non_red_numbers(json.loads(day(12)))

65402

## [Day 13](https://adventofcode.com/2015/day/13) Knights of the Dinner Table

We're asked to arrange people around a table to maximise happiness. A person's happiness is determined by who they're sat next to. 

In [40]:
example_13 = '''\
Alice would gain 54 happiness units by sitting next to Bob.
Alice would lose 79 happiness units by sitting next to Carol.
Alice would lose 2 happiness units by sitting next to David.
Bob would gain 83 happiness units by sitting next to Alice.
Bob would lose 7 happiness units by sitting next to Carol.
Bob would lose 63 happiness units by sitting next to David.
Carol would lose 62 happiness units by sitting next to Alice.
Carol would gain 60 happiness units by sitting next to Bob.
Carol would gain 55 happiness units by sitting next to David.
David would gain 46 happiness units by sitting next to Alice.
David would lose 7 happiness units by sitting next to Bob.
David would gain 41 happiness units by sitting next to Carol.
'''

First, let's get the map from pair to happiness.

In [41]:
def pairs_happiness(happiness_data):
    return {(w[0], w[-1].strip('.')): 
             int(w[3]) * (1 if w[2]=='gain' else -1)
        for w in map(str.split, happiness_data.splitlines())}

pairs_happiness(example_13)

{('Alice', 'Bob'): 54,
 ('Alice', 'Carol'): -79,
 ('Alice', 'David'): -2,
 ('Bob', 'Alice'): 83,
 ('Bob', 'Carol'): -7,
 ('Bob', 'David'): -63,
 ('Carol', 'Alice'): -62,
 ('Carol', 'Bob'): 60,
 ('Carol', 'David'): 55,
 ('David', 'Alice'): 46,
 ('David', 'Bob'): -7,
 ('David', 'Carol'): 41}

In [42]:
def table_happiness(seating_plan, pair_scores):
    N = len(seating_plan)
    return sum(pair_scores[(seating_plan[i      ], seating_plan[(i+1)%N])] +
               pair_scores[(seating_plan[(i+1)%N], seating_plan[i      ])]
               for i in range(N))

def best_plan(happiness_data):
    scores = pairs_happiness(happiness_data)
    people = {p for p, q in scores}
    return max(table_happiness(p, scores) 
               for p in its.permutations(people, len(people)))

assert best_plan(example_13) == 330
best_plan(day(13))

709

For part two, the host sits with the party -- having a score of 0 in any pair.

In [43]:
def best_plan_with_host(happiness_data):
    scores = pairs_happiness(happiness_data)
    people = {p for p, q in scores}
    scores.update({(p, 'host'): 0 for p in people})
    scores.update({('host', p): 0 for p in people})
    people.add('host')
    return max(table_happiness(p, scores) 
               for p in its.permutations(people, len(people)))

best_plan_with_host(day(13))

668

## [Day 14](https://adventofcode.com/2015/day/14) Reindeer Olympics

Part one asks us to calculate which reindeer is in the lead after a given time.

In [44]:
day_14='''\
Vixen can fly 19 km/s for 7 seconds, but then must rest for 124 seconds.
Rudolph can fly 3 km/s for 15 seconds, but then must rest for 28 seconds.
Donner can fly 19 km/s for 9 seconds, but then must rest for 164 seconds.
Blitzen can fly 19 km/s for 9 seconds, but then must rest for 158 seconds.
Comet can fly 13 km/s for 7 seconds, but then must rest for 82 seconds.
Cupid can fly 25 km/s for 6 seconds, but then must rest for 145 seconds.
Dasher can fly 14 km/s for 3 seconds, but then must rest for 38 seconds.
Dancer can fly 3 km/s for 16 seconds, but then must rest for 37 seconds.
Prancer can fly 25 km/s for 6 seconds, but then must rest for 143 seconds.
'''

example_14='''\
Comet can fly 14 km/s for 10 seconds, but then must rest for 127 seconds.
Dancer can fly 16 km/s for 11 seconds, but then must rest for 162 seconds.
'''

def distance(reindeer, T):
    'How far has the reindeer travelled after time T?'
    velocity, fly_t, rest_t = reindeer
    reps, rem = divmod(T, (fly_t + rest_t))
    return velocity * (reps * fly_t + min(rem, fly_t))

assert distance((14, 10, 127), 1000) == 1120
assert distance((16, 11, 162), 1000) == 1056

def winning_distance(reindeer_data, T):
    return max(distance(ints(s), T) for s in reindeer_data.splitlines())

winning_distance(day_14, 2503)

2660

Part two operates a different scoring system. At the end of each second the reindeer which has travelled furthest scores a point. 

In [45]:
def winning_score(reindeer_data, T):
    reindeer = [ints(s) for s in reindeer_data.splitlines()]
    scores = [0] * len(reindeer)
    for t in range(1, 1+T):
        _, i = max((distance(r, t), i) for i, r in enumerate(reindeer))
        scores[i] += 1
    return max(scores)
        
winning_score(example_14, 1000)

689

In [46]:
winning_score(day_14, 2503)

1256

## [Day 15](https://adventofcode.com/2015/day/15) Science for Hungry People

The question asks us to formulate a recipe using exactly 100 teaspoons of the supplied ingredients, maximising a score calculated from these ingredients.


In [47]:
input_15 = '''\
Frosting: capacity 4, durability -2, flavor 0, texture 0, calories 5
Candy: capacity 0, durability 5, flavor -1, texture 0, calories 8
Butterscotch: capacity -1, durability 0, flavor 5, texture 0, calories 6
Sugar: capacity 0, durability 0, flavor -2, texture 2, calories 1
'''
example_15 = '''\
Butterscotch: capacity -1, durability -2, flavor 6, texture 3, calories 8
Cinnamon: capacity 2, durability 3, flavor -2, texture -1, calories 3
'''

def recipe_score(measures, ingredients):
    N = len(ingredients[0])
    product = 1
    for i in range(N-1): # Note: Calories _not_ included in score
        score = sum(measure*ingredient[i] 
            for measure, ingredient in zip(measures, ingredients))
        product *= max(score, 0)
    return product

# Check the worked example
assert recipe_score([44,56], [ints(s) for s in example_15.splitlines()]) == 62842880

def partition(total, n):
    'Yield tuples of `n` numbers which sum to `total`'
    if n > total: 
        return
    elif n == 1:
        yield (total,)
    else:
        for i in range(1, total):
            yield from ((i, *r) for r in partition(total-i, n-1))

list(partition(5,3))

[(1, 1, 3), (1, 2, 2), (1, 3, 1), (2, 1, 2), (2, 2, 1), (3, 1, 1)]

That all looks reasonable. The question asks us to maximise the recipe score using measures of the supplied ingredients summing to 100.

In [48]:
def best_recipe(ingredients_data, total=100):
    ingredients = tuple(ints(s) for s in ingredients_data.splitlines())
    return max(recipe_score(measures, ingredients)
        for measures in partition(total, len(ingredients)))

assert best_recipe(example_15) == 62842880
best_recipe(input_15)

18965440

For part two, we're asked for the highest scoring cookie with a calorie count of 500.

In [49]:
def calorie_count(measures, ingredients):
    return sum(measure * ingredient[-1] 
               for measure, ingredient in zip(measures, ingredients))

def best_calorie_controlled_recipe(ingredients_data, total=100, cals=500):
    ingredients = tuple(ints(s) for s in ingredients_data.splitlines())
    return max(recipe_score(measures, ingredients)
               for measures in partition(total, len(ingredients))
               if calorie_count(measures, ingredients) == 500)

assert best_calorie_controlled_recipe(example_15) == 57600000
best_calorie_controlled_recipe(input_15)

15862900

## [Day 16](https://adventofcode.com/2015/day/16) Aunt Sue
We have to identify the "Sue" on the list which matches a supplied collections of characteristics.


In [50]:
match_16 = {
'children': 3,
'cats': 7,
'samoyeds': 2,
'pomeranians': 3,
'akitas': 0,
'vizslas': 0,
'goldfish': 5,
'trees': 3,
'cars': 2,
'perfumes': 1    
}

In [51]:
!head input/16

Sue 1: cars: 9, akitas: 3, goldfish: 0
Sue 2: akitas: 9, children: 3, samoyeds: 9
Sue 3: trees: 6, cars: 6, children: 4
Sue 4: trees: 4, vizslas: 4, goldfish: 9
Sue 5: akitas: 9, vizslas: 7, cars: 5
Sue 6: vizslas: 6, goldfish: 6, akitas: 3
Sue 7: pomeranians: 5, samoyeds: 0, perfumes: 10
Sue 8: cars: 10, pomeranians: 7, goldfish: 8
Sue 9: trees: 2, vizslas: 7, samoyeds: 6
Sue 10: perfumes: 5, pomeranians: 4, children: 9


In [52]:
def parse_sue(line):
    toks = [atom(w.rstrip(':,')) for w in line.split()]
    return toks[1], {toks[i]:toks[i+1] for i in range(2, len(toks), 2)}

# Let's find the Sues who share the same features as match_16
[i for i, features in map(parse_sue, day(16).splitlines())
if all(match_16[k] == v for k, v in features.items())]

[373]

For part two, the interpretation of `match_16` has changed.

In [53]:
def compatible(features, match):
    for f, v in features.items():
        if f in ('cats', 'trees'):
            if match[f] >= v: return False
        elif f in ('pomeranians', 'goldfish'):
            if match[f] <= v: return False
        else:
            if match[f] != v: return False
    return True
        
[i for i, features in map(parse_sue, day(16).splitlines())
if compatible(features, match_16)]

[260]

## [Day 17](https://adventofcode.com/2015/day/17) No Such Thing as Too Much

We are asked to combine containers so their total volume is a given number.

In [54]:
example_17 = [20,15,10,5,5]
containers_17 = [43,3,4,10,21,44,4,6,47,41,34,17,17,44,36,31,46,9,27,38]

def combos(containers, total):
    N = len(containers)
    return sum(1 for p in its.product([0,1], repeat=N) 
               if sum(map(operator.mul, p, containers)) == total)

assert combos(example_17, 25) == 4
combos(containers_17, 150)

1638

For Part Two, we want to use the minimum number of containers which give the required total. 

In [55]:
def container_counts(containers, total):
    N = len(containers)
    return collections.Counter(sum(p) for p in its.product([0,1], repeat=N) 
                   if sum(map(operator.mul, p, containers)) == total)

# Worked example. Check example_17 containers give a minimum no. of
# 2 containers, with 3 such combinations
assert min(container_counts(example_17, 25).items()) == (2, 3)

In [56]:
min(container_counts(containers_17, 150).items())

(4, 17)

## [Day 18](https://adventofcode.com/2015/day/18) Like a GIF For Your Yard
This is a cellular automaton, reminiscent of [Conway's Game of Life](http://wordaligned.org/life). We are asked to evolve a grid of lights, where each light's next state is based on its current state + the number of neighbours that are on.

In [57]:
def update_cell(grid, p):
    'Return the value taken by `grid` cell `p` in the next generation'
    v = grid[p]
    on_neighbours = sum(grid.get(n)=='#' for n in neighbours8(*p))
    return '#' if on_neighbours==3 or v=='#' and on_neighbours==2 else '.'

def evolve(grid, update_cell=update_cell):
    while True:
        yield grid
        grid = {p: update_cell(grid, p) for p in grid}

example_18='''\
.#.#.#
...##.
#....#
..#...
#.#..#
####..
'''

def load_grid(grid_data):
    return {(c,r):v 
            for r,line in enumerate(grid_data.splitlines()) 
            for c,v in enumerate(line)}

# Check the worked example, which shows 4 lights on after 4 steps
ex18_grid = load_grid(example_18)
assert sum(v=='#' for v in nth(evolve(ex18_grid), 4).values()) == 4
draw_grid(nth(evolve(ex18_grid), 4))

......
......
..##..
..##..
......
......


In [58]:
sum(v=='#' for v in nth(evolve(load_grid(day(18))), 100).values())

821

For part two, the corners of the grid are stuck on "on".

In [59]:
def is_corner(grid, p):
    'Return True iff p is at a corner of the grid'
    return sum(n in grid for n in neighbours4(*p)) == 2

def fixed_corners_update(grid, p):
    'A cell update function which leaves the corners lit'
    return '#' if is_corner(grid, p) else update_cell(grid, p)

def turn_corners_on(grid):
    return {p: '#' if is_corner(grid, p) else v for p, v in grid.items()}

ex18_grid = turn_corners_on(ex18_grid)
assert sum(v=='#' for v in nth(evolve(ex18_grid, fixed_corners_update), 5).values()) == 17
draw_grid(nth(evolve(ex18_grid, fixed_corners_update), 5))

##.###
.##..#
.##...
.##...
#.#...
##...#


In [60]:
grid18 = turn_corners_on(load_grid(day(18)))
sum(v=='#' for v in nth(evolve(grid18, fixed_corners_update), 100).values())

886

## [Day 19](https://adventofcode.com/2015/day/19): Medicine for Rudolph
A medicine molecule (represented as a string) evolves via replacement rules. The rules and the medicine molecule look like this.

In [61]:
!cat input/19

Al => ThF
Al => ThRnFAr
B => BCa
B => TiB
B => TiRnFAr
Ca => CaCa
Ca => PB
Ca => PRnFAr
Ca => SiRnFYFAr
Ca => SiRnMgAr
Ca => SiTh
F => CaF
F => PMg
F => SiAl
H => CRnAlAr
H => CRnFYFYFAr
H => CRnFYMgAr
H => CRnMgYFAr
H => HCa
H => NRnFYFAr
H => NRnMgAr
H => NTh
H => OB
H => ORnFAr
Mg => BF
Mg => TiMg
N => CRnFAr
N => HSi
O => CRnFYFAr
O => CRnMgAr
O => HP
O => NRnFAr
O => OTi
P => CaP
P => PTi
P => SiRnFAr
Si => CaSi
Th => ThCa
Ti => BP
Ti => TiTi
e => HF
e => NAl
e => OMg

ORnPBPMgArCaCaCaSiThCaCaSiThCaCaPBSiRnFArRnFArCaCaSiThCaCaSiThCaCaCaCaCaCaSiRnFYFArSiRnMgArCaSiRnPTiTiBFYPBFArSiRnCaSiRnTiRnFArSiAlArPTiBPTiRnCaSiAlArCaPTiTiBPMgYFArPTiRnFArSiRnCaCaFArRnCaFArCaSiRnSiRnMgArFYCaSiRnMgArCaCaSiThPRnFArPBCaSiRnMgArCaCaSiThCaSiRnTiMgArFArSiThSiThCaCaSiRnMgArCaCaSiRnFArTiBPTiRnCaSiAlArCaPTiRnFArPBPBCaCaSiThCaPBSiThPRnFArSiThCaSiThCaSiThCaPTiBSiRnFYFArCaCaPRnFArPBCaCaPBSiRnTiRnFArCaPRnFArSiRnCaCaCaSiThCaRnCaFArYCaSiRnFArBCaCaCaSiThFArPBFArCaSiRnFArRnCaCaCaFArSiRnFArTiRnPMgArF


In [62]:
def load_molecule_and_rules(data):
    'Parse question data, returning molecule and replacements'
    words = [line.split() for line in data.splitlines()]
    return words[-1][0], {(w[0], w[2]) for w in words if len(w)==3}

def replacements(molecule, src, dst):
    'Generate the results of applying src=>dst replacement to molecule'
    i = 0
    while i != -1:
        i = molecule.find(src, i)
        if i != -1:
            yield molecule[:i] + dst + molecule[i+len(src):]
            i += len(src)
    
def evolve_molecule(molecule, rules):
    'Evolve molecule using the supplied rules'
    molecules = set([molecule])
    while True:
        yield molecules
        molecules = {
            repl for rule in rules for molecule in molecules
            for repl in replacements(molecule, *rule)}

example_19 = '''\
H => HO
H => OH
O => HH

HOH
'''

# Check the worked example
molecules = evolve_molecule(*load_molecule_and_rules(example_19))
take(molecules, 3)

[{'HOH'},
 {'HHHH', 'HOHO', 'HOOH', 'OHOH'},
 {'HHHHO',
  'HHHOH',
  'HHOHH',
  'HOHHH',
  'HOHOO',
  'HOOHO',
  'HOOOH',
  'OHHHH',
  'OHOHO',
  'OHOOH',
  'OOHOH'}]

In [63]:
# Check the second worked example "HOHOHO" :-)
_, rules = load_molecule_and_rules(example_19)
assert len(nth(evolve_molecule('HOHOHO', rules), 1)) == 7

In [64]:
len(nth(evolve_molecule(*load_molecule_and_rules(day(19))), 1))

576

Part two introduces molecule fabrication. We start from the electron `e` and evolve until we reach the medicine molecule. How long does it take?

My initial attempt was to evolve the electron, but this didn't run quickly enough. My second idea was to reverse evolution and reduce the medicine molecule to the electron. This also took too long. My actual solution used the strategy of selecting random replacements, backing out when blocked. It looks very like there's only a single path from the input molecule to the target `"e"`.

In [65]:
example_19_part2 = '''\
e => H
e => O
H => HO
H => OH
O => HH

HOHOHO
'''

def fabrication_time(molecule, rules, seed='e'):
    '''How many steps to fabricate `molecule` from `seed`?
    
    - rules: the replacements
    '''
    rules = [(dst, src) for src, dst in rules] # Go backwards
    buf = collections.deque(maxlen=20)
    def blocked():
        return buf and max(buf) == 0
    
    while True:
        N = 0
        mol = molecule
        buf.clear()
        while mol != seed and not blocked():
            src, dst = random.choice(rules)
            mol, delta = re.subn(src, dst, mol)
            N += delta
            buf.append(delta)
        if mol == 'e':
            yield N

take(fabrication_time(*load_molecule_and_rules(day(19))), 10)

[207, 207, 207, 207, 207, 207, 207, 207, 207, 207]

## [Day 20](https://adventofcode.com/2015/day/20): Infinite Elves and Infinite Houses

Setup:
- an infinite street has houses numbered 1, 2, 3, ...
- there are also an infinite number of elves
- elf N delivers 10N presents to houses N, 2N, 3N, ...

We are asked for the lowest house number getting at least 36000000 presents.

We can divide by 10 for simplicity. 

First, try houses which are powers of 2.

House `2^N` gets `1 + 2 + 4 + ... 2^N` presents, i.e. `2^(N+1) - 1`


In [66]:
math.log(3600000, 2)

21.779565475879124

In [67]:
2**21, 2**22 - 1

(2097152, 4194303)

So, house `2**21` gets `41943030 - 1` presents. Are there any lower numbered houses  which get at least `36000000` presents?

In [68]:
street = [0]* 2**21
for elf in range(1, 2**21):
    street[elf::elf] = [v + elf for v in street[elf::elf]]

next(i for i, n in enumerate(street) if n*10 >= 36000000)

831600

For part two, each elf delivers 11 presents to a maximum of 50 homes.

In [69]:
street = [0]* 2**21
deliver50 = [1]*50 + [0]*2**21
for elf in range(1, 2**21):
    street[elf::elf] = [v + elf*f for v,f in zip(street[elf::elf],deliver50)]
    
next(i for i, n in enumerate(street) if n*11 >= 36000000)

884520

## [Day 21](https://adventofcode.com/2015/day/21): RPG Simulator 20XX
Player and boss take turns to fight. A participant wins when their opponent drops to 0 points.

In [70]:
armory_manifest='''\
Weapons:    Cost  Damage  Armor
Dagger        8     4       0
Shortsword   10     5       0
Warhammer    25     6       0
Longsword    40     7       0
Greataxe     74     8       0

Armor:      Cost  Damage  Armor
Leather      13     0       1
Chainmail    31     0       2
Splintmail   53     0       3
Bandedmail   75     0       4
Platemail   102     0       5

Rings:      Cost  Damage  Armor
Damage_1      25     1       0
Damage_2      50     2       0
Damage_3     100     3       0
Defense_1     20     0       1
Defense_2     40     0       2
Defense_3     80     0       3
'''

armor = collections.namedtuple('armor', 'cost damage armor')

def load_armory(data):
    result = {}
    for line in data.splitlines():
        if ':' in line:
            items = result.setdefault(line.partition(':')[0], [])
        elif line:
            items.append(armor(*[int(w) for w in line.split()[1:]]))
    return result

armory = load_armory(armory_manifest)
armory

{'Weapons': [armor(cost=8, damage=4, armor=0),
  armor(cost=10, damage=5, armor=0),
  armor(cost=25, damage=6, armor=0),
  armor(cost=40, damage=7, armor=0),
  armor(cost=74, damage=8, armor=0)],
 'Armor': [armor(cost=13, damage=0, armor=1),
  armor(cost=31, damage=0, armor=2),
  armor(cost=53, damage=0, armor=3),
  armor(cost=75, damage=0, armor=4),
  armor(cost=102, damage=0, armor=5)],
 'Rings': [armor(cost=25, damage=1, armor=0),
  armor(cost=50, damage=2, armor=0),
  armor(cost=100, damage=3, armor=0),
  armor(cost=20, damage=0, armor=1),
  armor(cost=40, damage=0, armor=2),
  armor(cost=80, damage=0, armor=3)]}

In [71]:
participant = collections.namedtuple('participant', 'hit_points damage armor')

def defeated(player):
    return player.hit_points <= 0

def battle(players, verbose=False):
    '''Run the battle to completion, returning the index of the winner
    
    - players: an array of two players
    '''
    for i,j in its.cycle([[0,1], [1,0]]):
        attacker, defender = players[i], players[j]
        damage = attacker.damage - defender.armor
        players[j] = defender._replace(hit_points=defender.hit_points-damage)
        if defeated(players[j]):
            return i
        if verbose:
            print(players[0].hit_points, players[1].hit_points)

# Run the worked example
battle([participant(8, 5, 5), participant(12, 7, 2)], verbose=True)

8 9
6 9
6 6
4 6
4 3
2 3


0

That looks good. Now we need to find the minimum spend for a win, given:
- an initial state for the boss 
- an initial number of hitpoints
- shopping rules

In [72]:
def combos(xs, n):
    for r in range(n+1):
        yield from its.combinations(xs, r)

def players(hitpts, armory):
    '''Generate the initial player configs
    
    - hitpoints: the fixed initial hitpoints
    - armory: the available Weapons, Armor, Rings
    
    yields a series of (participant, cost) pairs
    '''
    weapons = armory['Weapons'] # exactly one weapons
    arms    = combos(armory['Armor'], 1) # 0 or 1 armor
    rings   = combos(armory['Rings'], 2) # 0-2 rings
    for weapon, arm, ring in its.product(weapons, arms, rings):
            purchases = list(flatten(([weapon], arm, ring)))
            cost   = sum(p.cost for p in purchases)
            damage = sum(p.damage for p in purchases)
            armor  = sum(p.armor for p in purchases)
            yield participant(hitpts, damage, armor), cost

take(players(100, armory), 3)

[(participant(hit_points=100, damage=4, armor=0), 8),
 (participant(hit_points=100, damage=5, armor=0), 33),
 (participant(hit_points=100, damage=6, armor=0), 58)]

That all looks good. Now we can iterate over the players and find the minimum spend for the win.

In [73]:
boss = participant(109, 8, 2) # The initial boss state

min(cost for player, cost in players(100, armory) 
    if battle([player, boss]) == 0)

111

For part two, the question is what's the most we can spend but still lose?

In [74]:
max(cost for player, cost in players(100, armory) 
    if battle([player, boss]) == 1)

188

## [Day 22](https://adventofcode.com/2015/day/22): Wizard Simulator 20XX
 A combat game with subtle rules.
 
The spells differ in ways that make the code tricky: some have instant results, some have extended effects, and the shield spell takes effect when cast and when it expires. I also needed to prune the search space and game state to find the result.

In [75]:
# Inputs
boss_damage = 9  # The boss' attacks inflict this much damage
boss_points = 51 # The boss starts with this many points

# A representation of the state at some point in the game
game_state = collections.namedtuple('game_state',
    ['turn',                    # Whose turn to attack?
     'player_loss',             # (See part two) Points lost by player each turn
     'spent',                   # Mana spent by the player
     'points', 'armour', 'mana',# Points, armour and mana the player has
     'effects',                 # Dict mapping active effects to durations
     'boss_points'])            # Points the boss has


# Each spell accepts a game state and returns a new game state
def missile(g):    return g._replace(boss_points=g.boss_points - 4)
def drain(g):      return g._replace(points=g.points + 2, boss_points=g.boss_points - 2)
# Poison and recharge have the same effect each turn they are active
def poison(g,_):   return g._replace(boss_points=g.boss_points - 3)
def recharge(g,_): return g._replace(mana=g.mana + 101)
# The shield affects turns 6 and 1
def shield(g,d):   return g._replace(armour=g.armour + {1:-7, 6:7}.get(d, 0))

spell  = collections.namedtuple('spell', 'effect cost duration')
spells = (spell(missile,   53, 0),
          spell(drain,     73, 0),
          spell(shield,   113, 6),
          spell(poison,   173, 6),
          spell(recharge, 229, 5))

def process_effects(g):
    'Process active effects and return the resulting game state.'
    for effect, duration in g.effects.items():
        g = effect(g, duration)
    # Reduce the effect durations, removing expired effects
    return g._replace(effects={e:d-1 for e,d in g.effects.items() if d > 1})

def cast(g, spell):
    'Cast a spell and return the resulting game state.'
    g = g._replace(turn=boss,
                   spent=g.spent + spell.cost,
                   mana=g.mana - spell.cost,
                   effects=dict(g.effects)) # Note: copy the effects so the
                                            # input g's effects are not modified 
    if spell.duration == 0:
        g = spell.effect(g)
    else:
        g.effects[spell.effect] = spell.duration
    return g

def player(g):
    'The player takes a turn, yielding possible new game states.'
    g = g._replace(points=g.points - g.player_loss)
    if g.points > 0:
        for spell in spells:
            if spell.cost <= g.mana and spell.effect not in g.effects:
                yield cast(g, spell)

def boss(g):
    'The boss takes a turn, yielding the new game state.'
    damage = max(1, boss_damage - g.armour) # At least 1 damage done
    yield g._replace(turn=player, points=g.points - damage)

def play(game):
    'Play the game, returning the least amount of mana spent to win.'
    M = 1_000_000
    cheap_win = M

    def won(g):
        'Return True if this game state is a win for the player.'
        nonlocal cheap_win
        if g.boss_points <= 0:
            cheap_win = min(cheap_win, g.spent)
            return True

    def lost(g):
        'Return True if the player has lost.'
        return g.points <= 0

    def prune(g):
        'Return True if this game state can be pruned from the search.'
        nonlocal cheap_win
        return g.spent >= cheap_win # Don't continue if we've overspent

    while game:
        g = process_effects(game.pop())
        if not won(g):
            game.extend(g for g in g.turn(g)
                        if not (won(g) or lost(g) or prune(g)))

    return None if cheap_win == M else cheap_win

play([game_state(turn=player,
                 player_loss=0,
                 spent=0,
                 points=50,
                 armour=0,
                 mana=500,
                 boss_points=boss_points,
                 effects={})])

900

Part two adjusts the rules. You (the player) lose 1 hit point at the start of each turn you take. The question is, what it the least amount of mana you can spend and still win?

I added a field, `player_loss`, to the game state.

In [76]:
play([game_state(turn=player,
                 player_loss=1,
                 spent=0,
                 points=50,
                 armour=0,
                 mana=500,
                 boss_points=boss_points,
                 effects={})])

1216

## [Day 23](https://adventofcode.com/2015/day/23): Opening the Turing Lock
We have a computer program operating on registers `a` and `b`. The instructions are:
- `hlf r` sets register r to half its current value, then continues with the next instruction.
- `tpl r` sets register r to triple its current value, then continues with the next instruction.
- `inc r` increments register r, adding 1 to it, then continues with the next instruction.
- `jmp offset` is a jump; it continues with the instruction offset away relative to itself.
- `jie r`, offset is like jmp, but only jumps if register r is even ("jump if even").
- `jio r`, offset is like jmp, but only jumps if register r is 1 ("jump if one", not odd).

In [77]:
!head -5 input/23

jio a, +16
inc a
inc a
tpl a
tpl a


In [78]:
def instruction(s):
    args = s.split()
    val = int(args[-1]) if s.startswith('j') else 0
    return args[0], args[1].rstrip(','), val

def load_program(prog_data):
    return [instruction(s) for s in prog_data.splitlines()]

def run_program(instructions, regs, pc=0):
    while pc in range(0, len(instructions)):
        ins, reg, val = instructions[pc]
        if   ins == 'hlf': regs[reg] //= 2; pc += 1
        elif ins == 'tpl': regs[reg] *= 3 ; pc += 1
        elif ins == 'inc': regs[reg] += 1 ; pc += 1
        elif ins == 'jmp': pc += val
        elif ins == 'jie': pc += 1 if regs[reg]  % 2 else val
        elif ins == 'jio': pc += 1 if regs[reg] != 1 else val
        else: assert False
    return regs

example_23 = '''\
inc a
jio a, +2
tpl a
inc a
'''

run_program(load_program(example_23), {'a':0, 'b':0})

{'a': 2, 'b': 0}

In [79]:
run_program(load_program(day(23)), {'a':0, 'b':0})

{'a': 1, 'b': 170}

In [80]:
run_program(load_program(day(23)), {'a':1, 'b':0})

{'a': 1, 'b': 247}

## [Day 24](https://adventofcode.com/2015/day/24): It Hangs in the Balance

We are asked to load Santa's sleigh, subject to constraints:
- packages should be partitioned into 3 groups of equal weight
- the first group should have as few packages as possible
- the first group should additionally minimise the product of its weights


In [81]:
! sort -n input/24 | uniq -c

      1 1
      1 2
      1 3
      1 7
      1 11
      1 13
      1 17
      1 19
      1 23
      1 31
      1 37
      1 41
      1 43
      1 47
      1 53
      1 59
      1 61
      1 67
      1 71
      1 73
      1 79
      1 83
      1 89
      1 97
      1 101
      1 103
      1 107
      1 109
      1 113


Notice that the input weights are unique -- we'll code our solution around that assumption.

In [82]:
def product(xs):
    'Return the product of the numbers in `xs`'
    return functools.reduce(operator.mul, xs, 1)

def equal_sum_partitions(xs, n):
    ''' Return the ways of dividing set `xs` into n equal sum subsets.
    
    The results are ordered by subset length.
    '''
    T, m = divmod(sum(xs), n)
    assert m == 0
    if n == 1:
        yield [xs]
    else:
        for r in range(1, 1 + len(xs)//n):
            for L in its.combinations(xs, r):
                 if sum(L) == T:
                     rest = xs - set(L)
                     yield from ([L] + r for r in equal_sum_partitions(rest, n-1))


def quantum_entanglement(weights, n):
    _, ps = next(its.groupby(equal_sum_partitions(weights, n), 
                             key=lambda p: len(p[0])))
    return min(product(p[0]) for p in ps)

assert quantum_entanglement(set([1,2,3,4,5,7,8,9,10,11]), 3) == 99
%time quantum_entanglement(set(ints(day(24))), 3)

Wall time: 6min 24s


11846773891

Part two asks for the quantum entanglement when the parcels can be split 4 ways.

In [83]:
%time quantum_entanglement(set(ints(day(24))), 4)

Wall time: 1h 7min 14s


80393059

Ouch! But the job's done.

## [Day 25](https://adventofcode.com/2015/day/25): Let It Snow

A grid is filled diagonally in the order shown:

```
   | 1   2   3   4   5   6  
---+---+---+---+---+---+---+
 1 |  1   3   6  10  15  21
 2 |  2   5   9  14  20
 3 |  4   8  13  19
 4 |  7  12  18
 5 | 11  17
 6 | 16
 ```
 
 The first code is `20151125`. If N is the nth number, the next number will be `(N * 252533) % 33554393`.
 
 The puzzle input reads:
 
 > To continue, please consult the code grid in the manual.  Enter the code at row 2978, column 3083.

In [84]:
def grid_values():
    seed = 20151125
    for diagonal in its.count(1):
        for r in range(diagonal):
            yield diagonal - r, r + 1, seed
            seed = (seed * 252533) % 33554393
          
take(grid_values(), 10)

[(1, 1, 20151125),
 (2, 1, 31916031),
 (1, 2, 18749137),
 (3, 1, 16080970),
 (2, 2, 21629792),
 (1, 3, 17289845),
 (4, 1, 24592653),
 (3, 2, 8057251),
 (2, 3, 16929656),
 (1, 4, 30943339)]

Yes, that looks fine. Let's find the real answer.

In [85]:
next(v for r, c, v in grid_values() if r == 2978 and c == 3083)

2650453

![Completed calendar](calendar.png "Lovely snow scene")